In [ ]:
from utils.common import load_automata_definition
from pprint import pprint

In [ ]:
automata_def = "/Users/ralvarez20/Documents/development/automata/data/qrip_ex1.afd.txt"

In [ ]:
vocab, accepted_states, states_definition = load_automata_definition(automata_def)

In [ ]:
vocab, accepted_states, states_definition

In [ ]:
def print_table(header_data, states_def):
    str_header = "\t{}".format("\t".join(header_data))
    print(str_header)
    for key in states_def.keys():
        str_row = "{}".format(key)
        for v in header_data:
            if states_def[key].get(v):
                str_row += "\t{}".format(states_def[key].get(v))
            else:
                str_row += "\t"
        #transition_elements = list(states_def[key].values())
        #str_row = "{}\t{}".format(key, "\t".join(transition_elements))
        print(str_row)

In [ ]:
print_table(vocab, states_definition)

In [ ]:
# Agregar estado inicial y final
all_states = list(states_definition.keys())
states_definition["qstart"] = { "€": all_states[0] }
states_definition["qend"] = {}
final_states = accepted_states[2:].split(",")
for fstate in final_states:
    states_definition[fstate]["€"] = "qend"
all_states = list(states_definition.keys())
pprint(states_definition)

In [ ]:
states_definition_cpy = states_definition.copy()
node_elements = [ x for x in states_definition_cpy.keys() if x not in ["qstart", "qend"] ]
for original_state in node_elements:
    vocab = []
    for v in states_definition_cpy.values():
        vocab += list(v.keys())
    vocab = list(set(vocab))
    print_table(vocab, states_definition_cpy)
    print("\n")
    print("="*10)
    print("\n")
    out_trans = states_definition_cpy.pop(original_state)
    #print("Node to delete: {}".format(original_state))
    in_trans = []
    loops_trans = []
    # Agregar todas aquellas coincidentes con "Estado_Destino == Estado Actual" exceptuando los self-loops
    #print("Actual Table - Start")
    #pprint(states_definition_cpy)
    for ext_state in states_definition_cpy:
        tlist = states_definition_cpy[ext_state]
        for tlist_item in tlist:
            #print(tlist.get(tlist_item))
            if tlist.get(tlist_item) == original_state:
                if ext_state != original_state:
                    in_trans.append((ext_state, {tlist_item: tlist[tlist_item]}))
    
    # Realizar producto cruz de Incoming X Outgoing y agregarlos a la tabla de transiciones
    for inc_state, inc_transitions in in_trans:
        base_transition = list(inc_transitions.keys())[0]
        for out_state in out_trans:
            if out_trans[out_state] == original_state:
                if "+" in out_state:
                    base_transition += "({})*".format(out_state)
                else:
                    base_transition += "({}*)".format(out_state)
        for out_state in out_trans:
            if out_trans[out_state] != original_state:
                str_transition = "({}{})".format(base_transition, out_state)
                #print("{}[{}] --> {}".format(inc_state, str_transition, out_trans[out_state]))
                states_definition_cpy[inc_state][str_transition] = out_trans[out_state]
            
    # Unir caminos dobles
    for mod_state in states_definition_cpy:
        if mod_state not in ["qstart"]:
            # Unir mediante +
            items_to_merge = [ x for x in states_definition_cpy[mod_state] if states_definition_cpy[mod_state][x] == mod_state ]
            if len(items_to_merge) > 1:
                merged_trans = "+".join(items_to_merge)
                states_definition_cpy[mod_state][merged_trans] = mod_state
                for itm in items_to_merge:
                    del states_definition_cpy[mod_state][itm]
            
    # Una vez eliminado el estado, cambiar las referencias por 
    for origin_state, transition in in_trans:
        del states_definition_cpy[origin_state][list(transition.keys())[0]]
    
    qstart_elements = states_definition_cpy["qstart"]
    transitions_to_delete = []
    transitions_to_append = []
    for qe in qstart_elements.values():
        tvalues = [ x for x in qstart_elements.keys() if qstart_elements[x] == qe ]
        if len(tvalues) > 1:
            merged_trans = "(" + "+".join(tvalues) + ")"
            
            transitions_to_append.append((merged_trans, qe))
            #states_definition_cpy["qstart"][merged_trans] = qe
            transitions_to_delete += tvalues
            
    transitions_to_delete = list(set(transitions_to_delete))
    
    for tdel in transitions_to_delete:
        del states_definition_cpy["qstart"][tdel]
    
    for tapp in transitions_to_append:
        states_definition_cpy["qstart"][tapp[0]] = tapp[1]

    

In [ ]:
vocab = []
for v in states_definition_cpy.values():
    vocab += list(v.keys())
vocab = list(set(vocab))
print_table(vocab, states_definition_cpy)
print("\n")
print("="*10)
print("\n")

In [ ]:
qstart_keys = list(states_definition_cpy["qstart"].keys())

qstart_keys = sorted(qstart_keys)

regex = qstart_keys[0]


In [ ]:
regex

In [ ]:
regex.replace("€", "")